### List of tables:

1. [Table 1: Treatment table](#treatment)
2. [Table 2: Background characteristics for subjects](#balance)
3. [Table 3: Balance test](#balance_test)

#### Import modules

In [1]:
import os #filesystem support
import pandas as pd #dataframe 
import statsmodels.formula.api as smf  # regressions analysis

from statsmodels.iolib.summary2 import summary_col  #regression output table

#### Set project folders

In [2]:
PROJECT_FOLDER = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
FINAL_DATA_FOLDER = os.path.join(PROJECT_FOLDER, 'data', 'final')
TABLES_FOLDER = os.path.join(PROJECT_FOLDER, 'reports', 'tables')
FIGURES_FOLDER = os.path.join(PROJECT_FOLDER, 'reports', 'figures')

#### Set floating point format

In [3]:
pd.options.display.float_format = '{:.3f}'.format

#### Load dataframe

In [4]:
DATA =  os.path.join(FINAL_DATA_FOLDER, 'fair_advice_final.feather')
df = pd.read_feather(DATA)
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3598 entries, 0 to 3597
Data columns (total 24 columns):
Dataset                     3598 non-null int64
SubmitTime                  3598 non-null datetime64[ns]
Duration                    3598 non-null int64
Controlquestion             1799 non-null float64
Controlquestion_attempts    1799 non-null float64
Treatment                   3598 non-null int64
Treatment_name              3598 non-null object
Dictator                    3598 non-null int64
Decision                    3598 non-null int64
Advise                      3598 non-null int64
Follow                      1799 non-null float64
Belief1                     598 non-null float64
Belief2                     598 non-null float64
Age                         3598 non-null int64
Female                      3598 non-null int64
Education                   3598 non-null int64
Political                   3598 non-null int64
Trust_requester             1799 non-null float64
Motivatio

#### Select player X for further analysis

In [5]:
sel = (df.Dictator == 1)
df1 = df[(df.Dictator == 1)].sort_values('Treatment')

#### How often did workers try to answer the control question?

In [6]:
controlquestion = df1['Controlquestion_attempts']\
    .astype(int)\
    .value_counts()\
    .to_frame(name='Count')\
    .rename_axis('Attempts', axis=1)
display(controlquestion)

Attempts,Count
0,1797
1,2


#### Table 1: Treatment table <a id='treatment'></a>

In [7]:
index_second = pd.Index(['Dictate', 'Framed', 'Advise'], name='Treatment')

treat_table = pd.crosstab(index=df1['Treatment_first'], columns=df1['Treatment_second'], margins=True)\
    .reindex(index_second.append(pd.Index(['All'])), axis=1)\
    .rename_axis('', axis=0)

treat_table.to_latex(os.path.join(TABLES_FOLDER, 'treat_table.tex'), bold_rows=True, float_format="%.0f")
display(treat_table)

,Dictate,Framed,Advise,All
,,,,
No Risk,304,297,302,903
Risk,299,301,296,896
All,603,598,598,1799


#### Table 2:  Background characterisitcs of subjects <a id='balance'></a>

In [8]:
covariates = ['Age', 'Female', 'Education', 'Political']
pivot_index = ['Treatment_second', 'Treatment_first',]
balance = pd.pivot_table(df, index=pivot_index , aggfunc=['mean', 'sem'], values=covariates, margins=True)\
    .rename(columns={'mean':'Mean', 'sem':'Se'}, level=0)\
    .rename_axis(['Treatment', 'Option B'])\
    .swaplevel(axis=1)\
    .sort_index(axis=1, level=0)\
    .reindex(['Dictate', 'Framed', 'Advise', 'All'], axis=0, level=0)

balance.to_latex(
    os.path.join(TABLES_FOLDER, 'balance_table.tex'),
    bold_rows=True,
    float_format="%.2f"
)
display(balance)

Age       Education       Female       Political      
                     Mean    Se      Mean    Se   Mean    Se      Mean    Se
Treatment Option B                                                          
Dictate   No Risk  34.967 0.440     4.107 0.056  0.474 0.020    -0.488 0.045
          Risk     36.228 0.481     4.153 0.058  0.478 0.020    -0.420 0.047
Framed    No Risk  35.416 0.465     4.193 0.054  0.501 0.020    -0.463 0.045
          Risk     35.483 0.455     4.070 0.058  0.478 0.020    -0.352 0.046
Advise    No Risk  35.534 0.465     4.176 0.054  0.484 0.020    -0.428 0.046
          Risk     35.386 0.436     4.195 0.055  0.484 0.021    -0.491 0.045
All                35.502 0.187     4.149 0.023  0.483 0.008    -0.440 0.019

#### Table 3: Balance test <a id='balance_test'></a>

In [9]:
ols_results = []
for variable in covariates:
    reg = '{} ~ C(Treatment)'.format(variable)
    ols = smf.ols(formula=reg, data=df1).fit()
    ols = ols.get_robustcov_results()
    ols_results.append(ols)

auxiliary = {
    'N': lambda x: "{:d}".format(int(x.nobs)),
    'R2': lambda x: "{:.3f}".format(x.rsquared),
    'P>F': lambda x: "{:.3f}".format(x.f_pvalue.item()),
}

ols_balance = summary_col(
    ols_results,
    stars=True,
    info_dict=auxiliary,
    float_format='%.3f'
)

ols_balance = ols_balance\
    .tables[0]\
    .rename(mapper=lambda x: x.replace('C(Treatment)[T.', '').replace(']', ''), axis=0)\
    .rename({'Intercept':'Constant'})

ols_balance.to_latex(
    os.path.join(TABLES_FOLDER, 'ols_balance.tex'),
    bold_rows=True,
    float_format="%.3f"
)

display(ols_balance)

,Age,Female,Education,Political
Constant,35.224***,0.490***,4.109***,-0.444***
,(0.636),(0.029),(0.077),(0.062)
2,0.974,-0.015,0.192*,0.009
,(0.927),(0.041),(0.114),(0.092)
3,0.749,0.008,0.157,-0.074
,(0.930),(0.041),(0.106),(0.089)
4,0.899,0.008,0.044,0.115
,(0.915),(0.041),(0.112),(0.089)
5,-0.002,0.010,0.067,0.063
,(0.911),(0.041),(0.107),(0.092)
